In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data0=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data1=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data2=pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')
print(data0.info())
print(data0.head())
print(data1.info())
print(data1.head())
print(data2.info())
print(data2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
      id      

2. Train and test the model for each region:

In [ ]:
#2.1. Split the data into a training set and validation set at a ratio of 75:25.

data0_train, data0_valid=train_test_split(data0, test_size=0.25, random_state=12345)
data1_train, data1_valid=train_test_split(data1, test_size=0.25, random_state=12345)
data2_train, data2_valid=train_test_split(data2, test_size=0.25, random_state=12345)

#features and target for data0

features_train0=data0_train.drop(['product', 'id'], axis=1)
features_valid0=data0_valid.drop(['product', 'id'], axis=1)
target_train0=data0_train['product']
target_valid0=data0_valid['product']

#features and target for data1

features_train1=data1_train.drop(['product', 'id'], axis=1)
features_valid1=data1_valid.drop(['product', 'id'], axis=1)
target_train1=data1_train['product']
target_valid1=data1_valid['product']

#features and target for data2

features_train2=data2_train.drop(['product', 'id'], axis=1)
features_valid2=data2_valid.drop(['product', 'id'], axis=1)
target_train2=data2_train['product']
target_valid2=data2_valid['product']

#2.2. Train the model and make predictions for the validation set

#model training and prediction for data0
model0=LinearRegression()
model0.fit(features_train0, target_train0)

#model training and prediction for data1
model1=LinearRegression()
model1.fit(features_train1, target_train1)

#model training and prediction for data2
model2=LinearRegression()
model2.fit(features_train2, target_train2)

#2.3. Save the predictions and correct answers for the validation set.

predicted0=model0.predict(features_valid0)
mse0=mean_squared_error(target_valid0, predicted0)

predicted1=model1.predict(features_valid1)
mse1=mean_squared_error(target_valid1, predicted1)

predicted2=model2.predict(features_valid2)
mse2=mean_squared_error(target_valid2, predicted2)

#2.4. Print the average volume of predicted reserves and model RMSE.

print('region 0', ' ', round(mse0 ** 0.5, 2), ' ', round(predicted0.mean(),2))
print('region 1', ' ', round(mse1 ** 0.5, 2), ' ', round(predicted1.mean(),2))
print('region 2', ' ', round(mse2 ** 0.5, 2), ' ', round(predicted2.mean(),2))


region 0   37.58   92.59
region 1   0.89   68.73
region 2   40.03   94.97


2.5. Analyze the results.

region1 has rmse close to 0 which means the average values are distributed closely to mean and only differs by 0.89 units from the mean value. However, region0 and region2 has high rmse which implies that the values are widely distributed with an average deviation of 37 units from mean (region0) and an average deviation of 40 units from mean (region2). 

3. Prepare for profit calculation:

In [ ]:
#3.1. Store all key values for calculations in separate variables.

import numpy as np
cost_per_well=100000000/200
revenue_per_unit=4500

#3.2. Calculate the volume of reserves sufficient for developing a new well without losses. 

no_loss=cost_per_well/revenue_per_unit

#Compare the obtained value with the average volume of reserves in each region.

print('region0_avg',' ', np.ceil(data0['product'].mean()))
print('region1_avg',' ', np.ceil(data1['product'].mean()))
print('region2_avg',' ', np.ceil(data2['product'].mean()))
print('no_loss_units',' ', np.ceil(no_loss))

region0_avg   93.0
region1_avg   69.0
region2_avg   96.0
no_loss_units   112.0


3.3. Provide the findings about the preparation for profit calculation step.

Minimum 112 units (thousand barrels) are required to build a new well with no loss. In all the regions, the average no of units are lower than minimum required units for no_loss. 

4. Write a function to calculate profit from a set of selected oil wells and model predictions:

In [ ]:
#4.1. Pick the wells with the highest values of predictions.
#4.2. Summarize the target volume of reserves in accordance with these predictions

predicted0_df=pd.Series(predicted0)
predicted1_df=pd.Series(predicted1)
predicted2_df=pd.Series(predicted2)

target0=target_valid0.reset_index(drop=True)
target1=target_valid1.reset_index(drop=True)
target2=target_valid2.reset_index(drop=True)

def profit(target, predicted, count):
    predicted_sorted=predicted.sort_values(ascending=False)
    selected=target[predicted_sorted.index][:count]
    profit=selected.sum()*revenue_per_unit-(count*cost_per_well)
    return profit

#4.3. Provide findings: suggest a region for oil wells' development and justify the choice. 
#Calculate the profit for the obtained volume of reserves.

print('Profit Region 0:', round(profit(target0, predicted0_df, 200)/1000000,1),'USD million')
print('Profit Region 1:', round(profit(target1, predicted1_df, 200)/1000000,1),'USD million')
print('Profit Region 2:', round(profit(target2, predicted2_df, 200)/1000000,1),'USD million')
    

Profit Region 0: 33.2 USD million
Profit Region 1: 24.2 USD million
Profit Region 2: 27.1 USD million


Region 0 has the highest profit for selected top 200 wells. Hence Region 0 should be selected for building new wells.

5. Calculate risks and profit for each region:

In [ ]:
#5.1. Use the bootstrapping technique with 1000 samples to find the distribution of profit.

state=np.random.RandomState(12345)

#bootstraping for region0

values0=[]
for i in range(1000):
    predicted0_subsample=predicted0_df.sample(n=500, replace=True, random_state=state)
    target0_subsample=target0[predicted0_subsample.index]
    values0.append(profit(target0_subsample, predicted0_subsample,200))
values0=pd.Series(values0)

#bootstraping for region1

values1=[]
for i in range(1000):
    predicted1_subsample=predicted1_df.sample(n=500, replace=True, random_state=state)
    target1_subsample=target1[predicted1_subsample.index]
    values1.append(profit(target1_subsample, predicted1_subsample,200))
values1=pd.Series(values1)

#bootstraping for region2

values2=[]
for i in range(1000):
    predicted2_subsample=predicted2_df.sample(n=500, replace=True, random_state=state)
    target2_subsample=target2[predicted2_subsample.index]
    values2.append(profit(target2_subsample, predicted2_subsample,200))
values2=pd.Series(values2)

print('Region0:','|',round(values0.quantile(0.01)/1000000,1),'|', round(values0.quantile(0.99)/1000000,1))
print('Region1:','|',round(values1.quantile(0.01)/1000000,1),'|', round(values1.quantile(0.99)/1000000,1))
print('Region2:','|',round(values2.quantile(0.01)/1000000,1),'|', round(values2.quantile(0.99)/1000000,1))


Region0: | -2.1 | 10.2
Region1: | 0.5 | 10.3
Region2: | -2.1 | 10.4


In [ ]:
#5.2. Find average profit, 95% confidence interval and risk of losses. 
#Loss is negative profit, calculate it as a probability and then express as a percentage.

#Region0

count0=0
for i in range(len(values0)):
    if values0[i]<0:
        count0+=1
print('Region0:', 'Average Profit:',round(values0.mean()/1000000,1))
print('Region0:', '95% confidence interval:', round(values0.quantile(0.025)/1000000,1),'|', round(values0.quantile(0.975)/1000000,1))
print('Region0:', 'Risk of loss (%):', count0/len(values0)*100)

#Region1

count1=0
for i in range(len(values1)):
    if values1[i]<0:
        count1+=1
print('Region1:', 'Average Profit:',round(values1.mean()/1000000,1))
print('Region1:', '95% confidence interval:', round(values1.quantile(0.025)/1000000,1),'|', round(values1.quantile(0.975)/1000000,1))
print('Region1:', 'Risk of loss (%):', count1/len(values1)*100)

#region2 

count2=0
for i in range(len(values2)):
    if values2[i]<0:
        count2+=1
print('Region2:', 'Average Profit:',round(values2.mean()/1000000,1))
print('Region2:', '95% confidence interval:', round(values2.quantile(0.025)/1000000,1),'|', round(values2.quantile(0.975)/1000000,1))
print('Region2:', 'Risk of loss (%):', count2/len(values2)*100)


Region0: Average Profit: 4.3
Region0: 95% confidence interval: -1.0 | 9.5
Region0: Risk of loss (%): 6.0
Region1: Average Profit: 5.2
Region1: 95% confidence interval: 1.3 | 9.5
Region1: Risk of loss (%): 0.3
Region2: Average Profit: 4.2
Region2: 95% confidence interval: -1.2 | 9.9
Region2: Risk of loss (%): 6.2


5.3. Provide findings: suggest a region for development of oil wells and justify the choice.

Based on the average profit, confidence interval and risk % calculation, Region1 appears to be the best fit. Region1 has the lowest risk of only 0.3% which is much lower than 6% risk for rest of the 2 regions. The average profit for Region0 is also higher then rest of the regions with the 95% confidence interval value close to other regions.